In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-23 21:48:34--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.72MB/s    in 0.2s    

2021-06-23 21:48:34 (5.72 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Software_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42605767|R3EFW2STIYIY0I|B00MUTIDKI|     248732228|McAfee 2015 Inter...|        Software|          1|            2|          2|   N|                Y|I was very disapp...|I was very disapp...| 2015-08-31|
|         US|   51771800|R12NR0R5A9F7FT|B00EPACNUG|     531462352|Hallmark Card Stu...|        Software|          5|    

In [6]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3EFW2STIYIY0I|          1|            2|          2|   N|                Y|
|R12NR0R5A9F7FT|          5|            0|          0|   N|                Y|
|R1LSH74R9XAP59|          2|            0|          1|   N|                Y|
|R1QXUNTF76K7L6|          2|            0|          0|   N|                Y|
|R2F7DR75PS8NKT|          5|            0|          0|   N|                Y|
|R2C1DJSCC8UFS6|          3|            0|          0|   N|                Y|
|R1AXGS1W4YFXMX|          1|            0|          2|   N|                Y|
|R1XU1B93402SYJ|          1|            1|          1|   N|                Y|
|R2U432NB3OPVR0|          5|            0|          0|   N|                Y|
|R3R6FIMIOQ5SP9|          5|            0|          0|   N|     

In [7]:
# Count total vote
vine_df.count()

341931

In [33]:
#  Step 1: Filter DataFrame to retrieve all rows where total_votes count is greater than or equal to 20
total_votes_df = vine_df.filter("total_votes>=20").orderBy(vine_df["total_votes"].asc())
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3C3UU3AWSPCPQ|          1|           13|         20|   N|                N|
|R1VZNSTRV9IE38|          1|           20|         20|   N|                Y|
|R2M9V2XMOA5MEJ|          1|           19|         20|   N|                Y|
|R1QMF01BDRT2LJ|          1|           10|         20|   N|                N|
|R2G908XLS4JISS|          5|           20|         20|   N|                N|
|R3E9P8SZEMFTBK|          2|           12|         20|   N|                N|
|R3K2ZVAQ2O3LWT|          1|            7|         20|   N|                Y|
|R1CXK4SGLJB4FE|          1|           15|         20|   N|                Y|
|R3GJGFTSDNVCHH|          5|           16|         20|   N|                N|
|R3JANDU5B4OSF0|          2|            6|         20|   N|     

In [11]:
# Total_votes count is greater than or equal to 20
total_votes_df.count()

20216

In [12]:
# Step 2:  Filter filtered_df to retrieve rows where helpful_votes/total_votes is greater than or equal to 50%
helpful_votes_df = total_votes_df.filter((total_votes_df['helpful_votes']) / (total_votes_df['total_votes']) >= 0.5)
helpful_votes_df.show()
helpful_votes_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1PXUULGJ2ARI8|          3|           20|         20|   N|                Y|
|R2G908XLS4JISS|          5|           20|         20|   N|                N|
|R3QDRWZTGQPEDB|          1|           18|         20|   N|                Y|
|R1QMF01BDRT2LJ|          1|           10|         20|   N|                N|
|R1CXK4SGLJB4FE|          1|           15|         20|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



17762

In [26]:
# Step 3: Create table that only has the rows where a review was written as part of vine program(paid)
vine_program_paid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'Y')
vine_program_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1BGEGOX6SCYU2|          5|           19|         20|   Y|                N|
| RWQZCIUS2Z8IS|          5|           19|         20|   Y|                N|
|R3BCPKHRR5942R|          3|           19|         20|   Y|                N|
|R2ODPCF6F4LT3O|          3|           17|         20|   Y|                N|
|R2SLQR7YTQELLA|          5|           19|         20|   Y|                N|
| RPQBEIHH92962|          4|           17|         20|   Y|                N|
|R3SF7YZ9N8DVN2|          5|           19|         20|   Y|                N|
|R3125W32EF7TY8|          5|           20|         20|   Y|                N|
| RSWJ1R6T04O02|          5|           19|         20|   Y|                N|
|R3TCHJ0QACS445|          4|           19|         20|   Y|     

In [27]:
# Step 4: Create table that only has the rows where a review was not a part of vine program(unpaid)
vine_program_unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'N')
vine_program_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2G908XLS4JISS|          5|           20|         20|   N|                N|
| RF553TNOT43HO|          1|           19|         20|   N|                N|
|R3GJGFTSDNVCHH|          5|           16|         20|   N|                N|
|R17A3UL7L22B8B|          3|           18|         20|   N|                N|
| R1Q9Q3H7K7V90|          4|           19|         20|   N|                Y|
|R3E9P8SZEMFTBK|          2|           12|         20|   N|                N|
|R3R3MI19D1KJDF|          4|           20|         20|   N|                Y|
|R1QMF01BDRT2LJ|          1|           10|         20|   N|                N|
|R30TCX1Q46IFBX|          2|           20|         20|   N|                N|
|R3QDRWZTGQPEDB|          1|           18|         20|   N|     

In [16]:
# Step 5: a. Determine the total number of reviews
total_vine_reviews = vine_program_df.count()
print(f" There are total {total_vine_reviews} vine reviews")

 There are total 17514 vine reviews


In [21]:
# b. Number of 5-star reviews from vine program
five_star_vine_df = vine_program_df.filter(vine_program_df['star_rating'] == '5')
five_star_vine_reviews = five_star_vine_df.count()
print(f"There are total {five_star_vine_reviews} 5-star reviews from the vine program")

There are total 5154 5-star reviews from the vine program


In [25]:
# c. Total paid review
total_paid = vine_program_paid_df.count()
total_paid

248

In [28]:
# Total unpaid review
total_unpaid = vine_program_unpaid_df.count()
total_unpaid

17514

In [43]:
# 5-star review paid
five_paid = vine_program_paid_df.filter("star_rating=5").count()
print(f"There are total {five_paid} reviews are paid 5-star reviews")

There are total 102 reviews are paid 5-star reviews


In [44]:
# 5-star review unpaid
five_unpaid = vine_program_unpaid_df.filter("star_rating=5").count()
print(f"There are total {five_unpaid} reviews are unpaid 5-star reviews")

There are total 5154 reviews are unpaid 5-star reviews


In [46]:
# Percentage of 5-star review for paid
percentage = round(((five_paid / total_paid) * 100), 0)
print(f"There are {percentage}% of 5-star reviews are paid")

There are 41.0% of 5-star reviews are paid


In [47]:
# Percentage of 5-star review for unpaid
percentage = round(((five_unpaid / total_unpaid) * 100), 0)
print(f"There are {percentage}% of 5-star reviews are unpaid")

There are 29.0% of 5-star reviews are unpaid
